In [4]:
# pip install langchain openai

In [20]:
import pandas as pd, numpy as np, os, warnings, time, re
from tqdm import tqdm
tqdm.pandas()
from functools import partial

from langchain.prompts import PromptTemplate
from concurrent.futures import ThreadPoolExecutor
from openai import OpenAI

from sklearn.metrics import classification_report

# Connect to the GPT and read in the data

In [2]:
DATASET='election'

df = pd.read_csv(os.path.join(DATASET, "data_merged.csv"))

In [4]:
df = pd.read_csv(os.path.join("bias results",DATASET+"_"+openai_model+".csv"))

In [5]:
df.head()

,tweet_id,event,stance,full_text,context,train_stance,bias_cot_preds,task_preds,context_preds,cot_preds
0,657596473161457665,Donald Trump,agree,How can you vote for someone just because they...,The following statement is a social media post...,for,agree,disagree,agree,agree
1,657653572356743168,Donald Trump,disagree,@jimsciutto #GOP #DonaldTrump flicks his hair ...,The following statement is a social media post...,against,neutral,neutral,disagree,neutral
2,657766725358886912,Donald Trump,agree,#HillaryClinton learns &amp; retains new infor...,The following statement is a social media post...,for,agree,neutral,disagree,agree
3,657964576009424897,Donald Trump,agree,#Trump unleashed on MSM/Political Pundits is o...,The following statement is a social media post...,for,agree,agree,agree,agree
4,657970937849970688,Donald Trump,disagree,#DonaldTrump struggled to remain conscious dur...,The following statement is a social media post...,against,agree,neutral,disagree,agree


In [6]:
df.shape

(2378, 10)

In [7]:
with open("../aci_openai_credentials.txt", "r") as f:
    api_key = f.read().strip()
    
openai_model = "gpt-3.5-turbo"

# Define the Prompts

In [8]:
# task-only prompt

task_template = '''Classify the following statement as to whether it is for, against, or neutral. Only return the classification label for the statement, and no other text.
statement: {statement}
stance:'''

task_prompt = PromptTemplate(
    
    input_variables=["statement"],
    template=task_template
)

In [9]:
# context prompt

if DATASET == 'basil':
    context_template = '''The following statement is a news article that mentions the following entity. Classify the statement as to whether it is for, against, or neutral toward the following entity. Only return the stance classification of the statement toward the entity and no other text.
    entity: {event}
    statement: {statement}
    stance:'''

    context_prompt = PromptTemplate(
        input_variables=["event","statement"],
        template=context_template
    )
    
elif DATASET == 'election':
    context_template = '''The following statement is a social media post expressing an opinion about a politician. Classify the statement as to whether it is for, against, or neutral toward the following politcian. Only return the stance classification of the statement toward the politcian and no other text.
    politcian: {event}
    statement: {statement}
    stance:'''

    context_prompt = PromptTemplate(
        input_variables=["event","statement"],
        template=context_template
    )
    
elif DATASET == 'semeval':
    context_template = '''The following statement is a social media post expressing an opinion about the following entity. Classify the statement as to whether it is for, against, or neutral toward the following entity. Only return the stance classification of the statement toward the entity and no other text.
    entity: {event}
    statement: {statement}
    stance:'''

    context_prompt = PromptTemplate(
        input_variables=["event","statement"],
        template=context_template
    )

In [10]:
# zero-shot CoT prompt

if DATASET == 'basil':
    cot_template_1 = '''The following statement is a news article that mentions the following entity. Think step-by-step and explain the stance (for, against, or neutral) of the statement towards the entity.
    entity: {event}
    statement: {statement}
    explanation:'''

    cot_prompt_1 = PromptTemplate(
        input_variables=["event","statement"],
        template=cot_template_1
    )

    cot_template_2 ='''Therefore, based on your explanation, {stance_reason}, what is the final stance? only return the stance label as for, against, or neutral.
    entity: {event}
    statement: {statement}
    stance:'''

    cot_prompt_2 = PromptTemplate(
        input_variables=["event","statement","stance_reason"],
        template=cot_template_2
    )
    
elif DATASET == 'election':
    cot_template_1 = '''The following statement is a social media post expressing an opinion about the following politician. Think step-by-step and explain the stance (for, against, or neutral) of the statement towards the politician.
    politician: {event}
    statement: {statement}
    explanation:'''

    cot_prompt_1 = PromptTemplate(
        input_variables=["event","statement"],
        template=cot_template_1
    )

    cot_template_2 ='''Therefore, based on your explanation, {stance_reason}, what is the final stance? only return the stance label as for, against, or neutral.
    politician: {event}
    statement: {statement}
    stance:'''

    cot_prompt_2 = PromptTemplate(
        input_variables=["event","statement","stance_reason"],
        template=cot_template_2
    )
    
elif DATASET == 'semeval':
    cot_template_1 = '''The following statement is a social media post expressing an opinion about the following entity. Think step-by-step and explain the stance (for, against, or neutral) of the statement towards the entity.
    entity: {event}
    statement: {statement}
    explanation:'''

    cot_prompt_1 = PromptTemplate(
        input_variables=["event","statement"],
        template=cot_template_1
    )

    cot_template_2 ='''Therefore, based on your explanation, {stance_reason}, what is the final stance? only return the stance label as for, against, or neutral.
    entity: {event}
    statement: {statement}
    stance:'''

    cot_prompt_2 = PromptTemplate(
        input_variables=["event","statement","stance_reason"],
        template=cot_template_2
    )

In [11]:
# Bias aware CoT

if DATASET == 'basil':
    bias_template_1 = '''The following statement is a news article that mentions the following entity. Think step-by-step and explain the political bias (left, center, or right) of the statement towards the entity.
    entity: {event}
    statement: {statement}
    explanation:'''

    bias_prompt_1 = PromptTemplate(
        input_variables=["event","statement"],
        template=bias_template_1
    )

    bias_template_2 ='''Given your explanation of the politcial bias of the statement toward the entity, {bias_reason}, and based on your explanation, {stance_reason}, what is the final stance? only return the stance label as for, against, or neutral.
    entity: {event}
    statement: {statement}
    stance:'''

    bias_prompt_2 = PromptTemplate(
        input_variables=["event","statement","bias_reason", "stance_reason"],
        template=bias_template_2
    )
    
elif DATASET == 'election':
    bias_template_1 = '''The following statement is a social media post expressing an opinion about the following politician. Think step-by-step and explain the political bias (left, center, or right) of the statement towards the politcian.
    entity: {event}
    statement: {statement}
    explanation:'''

    bias_prompt_1 = PromptTemplate(
        input_variables=["event","statement"],
        template=bias_template_1
    )

    bias_template_2 ='''Given your explanation of the politcial bias of the statement toward the politcian, {bias_reason}, and based on your explanation, {stance_reason}, what is the final stance? only return the stance label as for, against, or neutral.
    entity: {event}
    statement: {statement}
    stance:'''

    bias_prompt_2 = PromptTemplate(
        input_variables=["event","statement","bias_reason", "stance_reason"],
        template=bias_template_2
    )
    
elif DATASET == 'semeval':
    bias_template_1 = '''The following statement is a social media post expressing an opinion about the following entity. Think step-by-step and explain the political bias (left, center, or right) of the statement towards the entity.
    entity: {event}
    statement: {statement}
    explanation:'''

    bias_prompt_1 = PromptTemplate(
        input_variables=["event","statement"],
        template=bias_template_1
    )

    bias_template_2 ='''Given your explanation of the politcial bias of the statement toward the entity, {bias_reason}, and based on your explanation, {stance_reason}, what is the final stance? only return the stance label as for, against, or neutral.
    entity: {event}
    statement: {statement}
    stance:'''

    bias_prompt_2 = PromptTemplate(
        input_variables=["event","statement","bias_reason", "stance_reason"],
        template=bias_template_2
    )

# Define the GPT Execution Functions

In [12]:
def run_openai_task(row):
    client = OpenAI(
        # This is the default and can be omitted
        api_key=api_key,
    )
    
    text = task_prompt.format(**{'statement':row['full_text']})
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": text
            }
        ],
        model=openai_model,
        max_tokens=10
    )
    label = chat_completion.dict()['choices'][0]['message']['content']
    
    return {'task_preds': label}

In [13]:
def run_openai_context(row):
    client = OpenAI(
        # This is the default and can be omitted
        api_key=api_key,
    )
    
    text = context_prompt.format(**{'statement':row['full_text'], 'event':row['event']})
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": text
            }
        ],
        model=openai_model,
        max_tokens=10
    )
    label = chat_completion.dict()['choices'][0]['message']['content']
    
    return {'context_preds': label}

In [14]:
def run_openai_cot(row):
    client = OpenAI(
        # This is the default and can be omitted
        api_key=api_key,
    )
    
    # run first prompt about explaining that it is biased
    text_1 = cot_prompt_1.format(**{'statement':row['full_text'], 'event':row['event']})
    chat_completion_1 = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": text_1
            }
        ],
        model=openai_model,
        max_tokens=100
    )
    stance_reason = chat_completion_1.dict()['choices'][0]['message']['content']
    
    text_2 = cot_prompt_2.format(**{'statement':row['full_text'], 'event':row['event'], 'stance_reason':stance_reason})
    chat_completion_2 = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": text_2
            }
        ],
        model=openai_model,
        max_tokens=10
    )
    label = chat_completion_2.dict()['choices'][0]['message']['content']
    
    return {'cot_preds': label}

In [15]:
def run_openai_bias_cot(row):
    client = OpenAI(
        # This is the default and can be omitted
        api_key=api_key,
    )
    
    # run first prompt about explaining that it is biased
    text_1 = cot_prompt_1.format(**{'statement':row['full_text'], 'event':row['event']})
    chat_completion_1 = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": text_1
            }
        ],
        model=openai_model,
        max_tokens=100
    )
    stance_reason = chat_completion_1.dict()['choices'][0]['message']['content']
    
    text_2 = bias_prompt_1.format(**{'statement':row['full_text'], 'event':row['event']})
    chat_completion_2 = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": text_2
            }
        ],
        model=openai_model,
        max_tokens=100
    )
    bias_reason = chat_completion_2.dict()['choices'][0]['message']['content']
    
    text_3 = bias_prompt_2.format(**{'statement':row['full_text'], 'event':row['event'], "bias_reason":bias_reason, "stance_reason":stance_reason})
    chat_completion_3 = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": text_3
            }
        ],
        model=openai_model,
        max_tokens=10
    )
    label = chat_completion_3.dict()['choices'][0]['message']['content']
    
    return {'bias_cot_preds': label}

# Run GPT Labeling

In [62]:
# run an example
row = df.iloc[5,:]

run_openai_task(row)

{'task_preds': 'for'}

In [63]:
run_openai_context(row)

{'context_preds': 'for'}

In [64]:
run_openai_cot(row)

{'cot_preds': 'for'}

In [65]:
run_openai_bias_cot(row)

{'bias_cot_preds': 'for'}

In [16]:
# Define a function to apply the chain to each row with retry mechanism
def apply_chain_with_retry(df, chain_function):
    results = []
    original_indices = [] 
    max_tries = 5
    i = 0
    for idx, row in tqdm(df.iterrows(), total=len(df), desc="Labeling by GPT"):
        tries = 0
        while tries < max_tries:
            try:
                # Invoke the chain for the current row
                output = chain_function(row)
                results.append(output)
                original_indices.append(idx)
                i +=1
                break  # Break out of the retry loop if successful
            except Exception as e:
                print(e)
                print(f"Waiting for 10 seconds before retrying row {i}...")
                tries +=1
                time.sleep(10)  # Wait for 10 seconds before retrying
    
    return pd.DataFrame(results, index=original_indices)

In [41]:
# Define the number of threads (adjust as needed)
num_threads = 3

# Split the DataFrame into chunks for parallel processing. Only take those rows that have not already been processed
chunks = np.array_split(df, num_threads)
#chunks = np.array_split(df[df['cot_preds'].isna()], num_threads)

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


### Run for Task-only Prompt

In [68]:
# Create a ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    # Apply the function to each chunk using concurrent futures
    partial_apply_chain = partial(apply_chain_with_retry, chain_function=run_openai_task)
    results = list(executor.map(partial_apply_chain, chunks))
    
task_output_df = pd.concat(results, axis=0)

Labeling by GPT:   0%|          | 0/793 [00:00<?, ?it/s]

Labeling by GPT:   0%|          | 0/792 [00:00<?, ?it/s]

Labeling by GPT:   0%|          | 1/793 [00:00<09:23,  1.40it/s]

Labeling by GPT:   0%|          | 2/793 [00:01<10:47,  1.22it/s]

Labeling by GPT:   0%|          | 2/793 [00:01<12:24,  1.06it/s]

Labeling by GPT:   1%|          | 4/793 [00:02<07:50,  1.68it/s]

Labeling by GPT:   1%|          | 6/793 [00:04<07:46,  1.69it/s]

Labeling by GPT:   1%|          | 7/793 [00:04<06:35,  1.99it/s]

Labeling by GPT:   1%|          | 7/792 [00:04<09:15,  1.41it/s]

Labeling by GPT:   1%|          | 8/793 [00:05<08:12,  1.59it/s]

Labeling by GPT:   1%|▏         | 10/793 [00:06<07:20,  1.78it/s]

Labeling by GPT:   1%|▏         | 11/793 [00:06<07:19,  1.78it/s]

Labeling by GPT:   2%|▏         | 12/793 [00:07<07:02,  1.85it/s]

Labeling by GPT:   2%|▏         | 13/793 [00:07<07:02,  1.85it/s]

Labeling by GPT:   2%|▏         | 13/793 [00:08<08:51,  1.47it/s]

Labeling by GPT:   2%

Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-vPUfp3M0Z7hDojWAmfplxq9Q on requests per day (RPD): Limit 10000, Used 10000, Requested 1. Please try again in 8.64s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}
Waiting for 10 seconds before retrying row 512...



Labeling by GPT:  97%|█████████▋| 771/793 [22:13<03:29,  9.51s/it]

Labeling by GPT:  97%|█████████▋| 772/793 [22:23<03:26,  9.84s/it]

Labeling by GPT:  65%|██████▍   | 514/792 [22:32<10:31:39, 136.33s/it]

Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-vPUfp3M0Z7hDojWAmfplxq9Q on requests per day (RPD): Limit 10000, Used 10000, Requested 1. Please try again in 8.64s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}
Waiting for 10 seconds before retrying row 772...




Labeling by GPT:  65%|██████▌   | 515/792 [22:43<7:35:22, 98.64s/it]  

Labeling by GPT:  65%|██████▌   | 516/792 [22:50<5:26:50, 71.05s/it]

Labeling by GPT:  98%|█████████▊| 774/793 [23:20<05:32, 17.51s/it]

...


Labeling by GPT:  99%|█████████▉| 787/793 [26:22<01:25, 14.30s/it]

Labeling by GPT:  66%|██████▋   | 526/792 [26:28<1:42:17, 23.07s/it]

Labeling by GPT:  67%|██████▋   | 527/792 [26:39<1:25:41, 19.40s/it]

Labeling by GPT:  67%|██████▋   | 528/792 [26:48<1:11:56, 16.35s/it]

Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-vPUfp3M0Z7hDojWAmfplxq9Q on requests per day (RPD): Limit 10000, Used 10000, Requested 1. Please try again in 8.64s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}
Waiting for 10 seconds before retrying row 787...




Labeling by GPT:  99%|█████████▉| 789/793 [27:08<01:10, 17.58s/it]
KeyboardInterrupt



Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-vPUfp3M0Z7hDojWAmfplxq9Q on requests per day (RPD): Limit 10000, Used 10000, Requested 1. Please try again in 8.64s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}
Waiting for 10 seconds before retrying row 685...
Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-vPUfp3M0Z7hDojWAmfplxq9Q on requests per day (RPD): Limit 10000, Used 10000, Requested 1. Please try again in 8.64s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}
Waiting for 10 seconds before retrying row 529...



Labeling by GPT: 100%|█████████▉| 790/793 [27:19<00:06,  2.08s/it]

In [79]:
#df.update(task_output_df)
df = df.merge(task_output_df, how='left', left_index=True, right_index=True)

In [66]:
df[df['task_preds'].isna()]

,tweet_id,event,stance,full_text,context,train_stance,bias_cot_preds,task_preds,context_preds,cot_preds


In [22]:
df.head()

,tweet_id,event,stance,full_text,context,train_stance,bias_cot_preds,task_preds,context_preds,cot_preds
0,657596473161457665,Donald Trump,agree,How can you vote for someone just because they...,The following statement is a social media post...,for,disagree,against,for,NaN
1,657653572356743168,Donald Trump,disagree,@jimsciutto #GOP #DonaldTrump flicks his hair ...,The following statement is a social media post...,against,neutral,Neutral,against,NaN
2,657766725358886912,Donald Trump,agree,#HillaryClinton learns &amp; retains new infor...,The following statement is a social media post...,for,disagree,Neutral,against,NaN
3,657964576009424897,Donald Trump,agree,#Trump unleashed on MSM/Political Pundits is o...,The following statement is a social media post...,for,agree,For,for,NaN
4,657970937849970688,Donald Trump,disagree,#DonaldTrump struggled to remain conscious dur...,The following statement is a social media post...,against,disagree,neutral,against,NaN


In [82]:
df.to_csv(os.path.join("bias results",DATASET+"_"+openai_model+".csv"), index=False)

### Run for Context Prompt

In [ ]:
# Create a ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    # Apply the function to each chunk using concurrent futures
    partial_apply_chain = partial(apply_chain_with_retry, chain_function=run_openai_context)
    results = list(executor.map(partial_apply_chain, chunks))
    
context_output_df = pd.concat(results, axis=0)

Labeling by GPT:  65%|██████▌   | 518/793 [3:25:10<1:06:33, 14.52s/it]

Labeling by GPT:  60%|██████    | 478/792 [3:25:19<2:01:42, 23.26s/it]

Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-vPUfp3M0Z7hDojWAmfplxq9Q on requests per day (RPD): Limit 10000, Used 10000, Requested 1. Please try again in 8.64s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}
Waiting for 10 seconds before retrying row 515...




Labeling by GPT:  60%|██████    | 479/792 [3:25:28<1:39:55, 19.15s/it]

In [85]:
#df.update(context_output_df)
df = df.merge(context_output_df, how='left', left_index=True, right_index=True)

In [ ]:
df[df['context_preds'].isna()]

In [ ]:
df.head()

In [88]:
df.to_csv(os.path.join("bias results",DATASET+"_"+openai_model+".csv"), index=False)

### Run for CoT Prompt

In [32]:
# Create a ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    # Apply the function to each chunk using concurrent futures
    partial_apply_chain = partial(apply_chain_with_retry, chain_function=run_openai_cot)
    results = list(executor.map(partial_apply_chain, chunks))
    
cot_output_df = pd.concat(results, axis=0)


Labeling by GPT:   0%|          | 0/40 [00:00<?, ?it/s]

Labeling by GPT:   0%|          | 0/39 [00:00<?, ?it/s]

Labeling by GPT:   2%|▎         | 1/40 [00:02<01:52,  2.90s/it]

Labeling by GPT: 100%|██████████| 40/40 [01:21<00:00,  2.03s/it]


Labeling by GPT:   8%|▊         | 3/39 [10:07<2:45:52, 276.47s/it]

Labeling by GPT:  10%|█         | 4/39 [10:26<1:42:08, 175.11s/it]

Labeling by GPT:  13%|█▎        | 5/39 [10:28<1:03:50, 112.67s/it]

Labeling by GPT:  15%|█▌        | 6/39 [10:31<41:21, 75.18s/it]   

Labeling by GPT:  18%|█▊        | 7/39 [10:32<27:15, 51.10s/it]

Labeling by GPT:  21%|██        | 8/39 [10:34<18:14, 35.30s/it]

Labeling by GPT:  23%|██▎       | 9/39 [10:35<12:21, 24.73s/it]

Labeling by GPT:  26%|██▌       | 10/39 [10:37<08:33, 17.71s/it]

Labeling by GPT:  28%|██▊       | 11/39 [10:39<06:03, 12.97s/it]

Labeling by GPT:  31%|███       | 12/39 [10:42<04:24,  9.81s/it]

Labeling by GPT:  33%|███▎      | 13/39 [10:44<03:12,  7.39s/it]

Labeling by GPT:  36%|

In [33]:
df.update(cot_output_df)
#df = df.merge(cot_output_df, how='left', left_index=True, right_index=True)

In [34]:
df[df['cot_preds'].isna()]

,tweet_id,event,stance,full_text,context,train_stance,bias_cot_preds,task_preds,context_preds,cot_preds


In [35]:
df.head()

,tweet_id,event,stance,full_text,context,train_stance,bias_cot_preds,task_preds,context_preds,cot_preds
0,657596473161457665,Donald Trump,agree,How can you vote for someone just because they...,The following statement is a social media post...,for,agree,disagree,agree,Against
1,657653572356743168,Donald Trump,disagree,@jimsciutto #GOP #DonaldTrump flicks his hair ...,The following statement is a social media post...,against,neutral,neutral,disagree,neutral
2,657766725358886912,Donald Trump,agree,#HillaryClinton learns &amp; retains new infor...,The following statement is a social media post...,for,agree,neutral,disagree,against
3,657964576009424897,Donald Trump,agree,#Trump unleashed on MSM/Political Pundits is o...,The following statement is a social media post...,for,agree,agree,agree,for
4,657970937849970688,Donald Trump,disagree,#DonaldTrump struggled to remain conscious dur...,The following statement is a social media post...,against,agree,neutral,disagree,against


In [30]:
df.to_csv(os.path.join("bias results",DATASET+"_"+openai_model+".csv"), index=False)

### Run for Bias Aware CoT

In [ ]:
# Create a ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=num_threads) as executor:
    # Apply the function to each chunk using concurrent futures
    partial_apply_chain = partial(apply_chain_with_retry, chain_function=run_openai_bias_cot)
    results = list(executor.map(partial_apply_chain, chunks))
    
bias_cot_output_df = pd.concat(results, axis=0)



Labeling by GPT:   0%|          | 1/793 [00:03<48:49,  3.70s/it]

Labeling by GPT:   0%|          | 1/792 [00:03<50:52,  3.86s/it]

Labeling by GPT:   0%|          | 2/793 [00:07<53:11,  4.03s/it]

Labeling by GPT:   1%|          | 4/793 [00:15<50:16,  3.82s/it]

Labeling by GPT:   1%|          | 5/793 [00:18<48:18,  3.68s/it]

Labeling by GPT:   1%|          | 6/793 [00:22<50:33,  3.85s/it]

Labeling by GPT:   1%|          | 7/793 [00:26<48:06,  3.67s/it]

Labeling by GPT:   1%|          | 8/793 [00:30<49:29,  3.78s/it]

Labeling by GPT:   1%|          | 9/793 [00:33<50:27,  3.86s/it]

Labeling by GPT:   1%|▏         | 10/793 [00:37<49:20,  3.78s/it]

Labeling by GPT:   1%|▏         | 11/793 [00:40<47:10,  3.62s/it]

Labeling by GPT:   2%|▏         | 12/793 [00:43<44:25,  3.41s/it]

Labeling by GPT:   2%|▏         | 13/793 [00:48<46:14,  3.56s/it]

Labeling by GPT:   2%|▏         | 14/793 [00:51<45:57,  3.54s/it]

Labeling by GPT:   2%|▏         | 15/793 [00:55<48:43,  3.76s/it]

La

Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-vPUfp3M0Z7hDojWAmfplxq9Q on requests per day (RPD): Limit 10000, Used 10000, Requested 1. Please try again in 8.64s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}
Waiting for 10 seconds before retrying row 116...
Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-vPUfp3M0Z7hDojWAmfplxq9Q on requests per day (RPD): Limit 10000, Used 10000, Requested 1. Please try again in 8.64s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}
Waiting for 10 seconds before retrying row 88...



Labeling by GPT:  15%|█▍        | 117/793 [09:32<5:48:41, 30.95s/it]

Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-vPUfp3M0Z7hDojWAmfplxq9Q on requests per day (RPD): Limit 10000, Used 10000, Requested 1. Please try again in 8.64s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}
Waiting for 10 seconds before retrying row 88...
Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-vPUfp3M0Z7hDojWAmfplxq9Q on requests per day (RPD): Limit 10000, Used 10000, Requested 1. Please try again in 8.64s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}
Waiting for 10 seconds before retrying row 117...




Labeling by GPT:  11%|█         | 89/792 [10:14<8:00:38, 41.02s/it]

...

Labeling by GPT:  39%|███▉      | 310/793 [8:08:14<12:19:09, 91.82s/it] 

Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-vPUfp3M0Z7hDojWAmfplxq9Q on requests per day (RPD): Limit 10000, Used 10000, Requested 1. Please try again in 8.64s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}
Waiting for 10 seconds before retrying row 313...
Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-vPUfp3M0Z7hDojWAmfplxq9Q on requests per day (RPD): Limit 10000, Used 10000, Requested 1. Please try again in 8.64s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}
Waiting for 10 seconds before retrying row 338...



Labeling by GPT:  46%|████▌     | 365/793 [8:08:33<12:58:20, 109.11s/it]

Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-vPUfp3M0Z7hDojWAmfplxq9Q on requests per day (RPD): Limit 10000, Used 10000, Requested 1. Please try again in 8.64s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}
Waiting for 10 seconds before retrying row 260...
Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-vPUfp3M0Z7hDojWAmfplxq9Q on requests per day (RPD): Limit 10000, Used 10000, Requested 1. Please try again in 8.64s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}
Waiting for 10 seconds before retrying row 338...
Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-3.5-turbo in organization org-vPUfp3M0Z7hDojWAmfplxq9Q on requests per day (RPD): Limit 10000, Used 10000, Requested 1. Please



Labeling by GPT:  48%|████▊     | 379/792 [8:10:01<13:07:22, 114.39s/it]

In [ ]:
bias_cot_output_df.shape

In [54]:
df.update(bias_cot_output_df)
#df = df.merge(bias_cot_output_df, how='left', left_index=True, right_index=True)

In [55]:
df[df['bias_cot_preds'].isna()]

,tweet_id,event,stance,full_text,context,train_stance,task_preds,context_preds,cot_preds,bias_cot_preds


In [56]:
df.head()

,tweet_id,event,stance,full_text,context,train_stance,task_preds,context_preds,cot_preds,bias_cot_preds
0,657596473161457665,Donald Trump,agree,How can you vote for someone just because they...,The following statement is a social media post...,for,disagree,agree,disagree,Against
1,657653572356743168,Donald Trump,disagree,@jimsciutto #GOP #DonaldTrump flicks his hair ...,The following statement is a social media post...,against,neutral,disagree,neutral,Neutral
2,657766725358886912,Donald Trump,agree,#HillaryClinton learns &amp; retains new infor...,The following statement is a social media post...,for,neutral,disagree,disagree,Against
3,657964576009424897,Donald Trump,agree,#Trump unleashed on MSM/Political Pundits is o...,The following statement is a social media post...,for,agree,agree,agree,for
4,657970937849970688,Donald Trump,disagree,#DonaldTrump struggled to remain conscious dur...,The following statement is a social media post...,against,neutral,disagree,disagree,Against


In [57]:
df.to_csv(os.path.join("bias results",DATASET+"_"+openai_model+".csv"), index=False)

### Post Process and Save out the Results!

In [36]:
def recode_stance(label):
    label = label.lower()  # Convert to lowercase for case-insensitive matching
    
    if 'for' in label or 'agree' in label:
        return 'agree'
    elif 'against' in label or 'disagree' in label:
        return 'disagree'
    elif 'neutral' in label:
        return 'neutral'

In [37]:
for column in ['task_preds', 'context_preds', 'cot_preds', 'bias_cot_preds']:
    df[column] = df[column].map(recode_stance)

In [38]:
np.unique(df[['task_preds', 'context_preds', 'cot_preds', 'bias_cot_preds']], return_counts=True)

(array(['agree', 'disagree', 'neutral'], dtype=object),
 array([6300, 1641, 1571]))

In [39]:
df.head()

,tweet_id,event,stance,full_text,context,train_stance,bias_cot_preds,task_preds,context_preds,cot_preds
0,657596473161457665,Donald Trump,agree,How can you vote for someone just because they...,The following statement is a social media post...,for,agree,agree,agree,disagree
1,657653572356743168,Donald Trump,disagree,@jimsciutto #GOP #DonaldTrump flicks his hair ...,The following statement is a social media post...,against,neutral,neutral,agree,neutral
2,657766725358886912,Donald Trump,agree,#HillaryClinton learns &amp; retains new infor...,The following statement is a social media post...,for,agree,neutral,agree,disagree
3,657964576009424897,Donald Trump,agree,#Trump unleashed on MSM/Political Pundits is o...,The following statement is a social media post...,for,agree,agree,agree,agree
4,657970937849970688,Donald Trump,disagree,#DonaldTrump struggled to remain conscious dur...,The following statement is a social media post...,against,agree,neutral,agree,disagree


In [40]:
df.to_csv(os.path.join("bias results",DATASET+"_"+openai_model+".csv"), index=False)